In [1]:
import pandas as pd
from footix.models.bayesian import Bayesian
import footix.implied_odds as odds
import collections
from footix.data_io.data_scrapper import ScrapFootballData


In [2]:
dataset = ScrapFootballData(competition="ligue2", season="2425", path ="./data", force_reload=True).get_data()

In [3]:
dataset.head(-5)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA
0,F2,16/08/2024,19:00,Ajaccio,Rodez,1,0,H,0,0,...,1.93,1.93,1.97,1.92,2.02,1.96,1.96,1.89,2.00,1.99
1,F2,16/08/2024,19:00,Amiens,Red Star,3,0,H,1,0,...,2.13,1.75,2.17,1.76,2.17,1.84,2.07,1.76,2.19,1.83
2,F2,16/08/2024,19:00,Clermont,Pau FC,2,2,D,1,2,...,1.83,2.03,1.86,2.03,1.89,2.10,1.80,2.02,1.89,2.10
3,F2,16/08/2024,19:00,Dunkerque,Annecy,0,2,A,0,0,...,1.78,2.10,1.75,2.16,1.83,2.22,1.72,2.13,1.82,2.20
4,F2,16/08/2024,19:00,Grenoble,Laval,2,1,H,0,0,...,2.00,1.85,2.01,1.88,2.02,1.92,1.97,1.84,2.05,1.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,F2,07/04/2025,19:45,Dunkerque,Guingamp,3,1,H,0,0,...,1.99,1.74,2.07,1.83,2.09,1.83,2.01,1.80,2.11,1.89
261,F2,11/04/2025,19:00,Amiens,Dunkerque,1,0,H,0,0,...,1.98,1.88,1.98,1.91,2.06,1.91,1.93,1.87,2.05,1.94
262,F2,11/04/2025,19:00,Martigues,Metz,1,4,A,1,3,...,1.83,2.03,1.85,2.04,1.85,2.07,1.81,2.00,1.83,2.15
263,F2,11/04/2025,19:00,Pau FC,Clermont,2,2,D,1,1,...,2.00,1.85,2.00,1.89,2.03,1.89,1.96,1.84,2.09,1.90


In [4]:
dataset["HomeTeam"].unique()

array(['Ajaccio', 'Amiens', 'Clermont', 'Dunkerque', 'Grenoble',
       'Guingamp', 'Caen', 'Martigues', 'Metz', 'Annecy', 'Bastia',
       'Laval', 'Paris FC', 'Pau FC', 'Rodez', 'Troyes', 'Lorient',
       'Red Star'], dtype=object)

In [5]:
model = Bayesian(n_teams=18, n_goals=15)

In [6]:
model.fit(X_train=dataset)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (6 chains in 6 jobs)
NUTS: [home, intercept, tau_att, raw_atts, tau_def, raw_defs]


Output()

Sampling 6 chains for 500 tune and 2_000 draw iterations (3_000 + 12_000 draws total) took 10 seconds.


In [ ]:
fixture = pd.read_csv("ligue2_18-04-2025.csv")
# Assuming fixture and odds are already defined DataFrames
for idx, match in fixture.iterrows():
    print("#" * 50)
    print(f"{match['home_team']:<30} {match['away_team']:<30}")
    # Display probabilities from Poisson prediction
    probas = model.predict(home_team=match["home_team"], away_team=match["away_team"]).return_probas()
    home_prob = probas[0]
    draw_prob = probas[1]
    away_prob = probas[2]
    print(f"  Probabilities: Home: {home_prob:.2f}, Draw: {draw_prob:.2f}, Away: {away_prob:.2f}")
    # Display odds from the function
    odds_list = [match["H"], match["D"], match["A"]]
    odds_result = odds.shin(odds=odds_list)[0]
    print(f"  Odds: Home: {odds_result[0]:.2f}, Draw: {odds_result[1]:.2f}, Away: {odds_result[2]:.2f}")
    print("#" * 50)

##################################################
AC Ajaccio                     Pau                           


ValueError: y contains previously unseen labels: 'AC Ajaccio'

In [19]:
#pm.summary(model.trace)

In [20]:
# Assuming fixture and odds are already defined DataFrames
fixture = pd.read_csv("odds_info.csv")
selections = []

for idx, match in fixture.iterrows():
    home_team = match["home_team"]
    away_team = match["away_team"]
    name = f"{home_team} vs {away_team} - "

    # Display probabilities from Poisson prediction
    probas = model.predict(home_team=home_team, away_team=away_team).return_probas()
    home_prob = probas[0]
    draw_prob = probas[1]
    away_prob = probas[2]

    # Display odds from the function
    odds_list = [match["H"], match["D"], match["A"]]
    odds_result = odds.shin(odds=odds_list)[0]
    home_implied = odds_result[0]
    draw_implied = odds_result[1]
    away_implied = odds_result[2]

    home_odd = odds_list[0]
    draw_odd = odds_list[1]
    away_odd = odds_list[2]

    # Calculate the value for each outcome
    home_value = home_prob - home_implied
    draw_value = draw_prob - draw_implied
    away_value = away_prob - away_implied

    # Determine the best selection
    if home_value >= draw_value and home_value >= away_value:
        best_prob = home_prob
        best_odd = home_odd
        sel = "home"
    elif draw_value >= home_value and draw_value >= away_value:
        best_prob = draw_prob
        best_odd = draw_odd
        sel = "draw"
    else:
        best_prob = away_prob
        best_odd = away_odd
        sel = "away"
    # Append the selection to the list
    selections.append({
        "name": name+sel,
        "probability": best_prob,
        "odds_bookie": best_odd
    })

In [21]:
print(selections)

[{'name': 'Rennes vs Nantes - away', 'probability': 0.30277683589466087, 'odds_bookie': 4.45}, {'name': 'Paris SG vs Le Havre - away', 'probability': 0.06464313101722349, 'odds_bookie': 16.0}, {'name': 'Monaco vs Strasbourg - away', 'probability': 0.21752235242031678, 'odds_bookie': 5.1}, {'name': 'Marseille vs Montpellier - draw', 'probability': 0.1383516745906119, 'odds_bookie': 7.0}, {'name': 'Lille vs Auxerre - away', 'probability': 0.23505071892226573, 'odds_bookie': 6.2}, {'name': 'Nice vs Angers - away', 'probability': 0.16377698966446522, 'odds_bookie': 7.6}, {'name': 'Brest vs Lens - away', 'probability': 0.31748628634625736, 'odds_bookie': 3.0}, {'name': 'Reims vs Toulouse - away', 'probability': 0.4717547202016954, 'odds_bookie': 2.1}, {'name': 'St Etienne vs Lyon - away', 'probability': 0.6159054186063452, 'odds_bookie': 1.55}, {'name': 'Nantes vs Paris SG - away', 'probability': 0.7438752173474351, 'odds_bookie': 1.33}, {'name': 'Paris SG vs Nice - away', 'probability': 0.

In [22]:
from footix.strategy.strategies import realKelly

In [23]:
realKelly(selections[:9], bankroll=158, max_multiple=3, device="cpu", early_stopping=False, num_iterations=5_000)

  0%|          | 0/5000 [00:00<?, ?it/s]


2025-04-16 19:31:10- Optimization finished. Runtime --- 7.568 seconds ---

Objective: -5.10269
Certainty Equivalent: 164.464

Rennes vs Nantes - away @ 4.45- € 14
Monaco vs Strasbourg - away @ 5.1- € 3
Lille vs Auxerre - away @ 6.2- € 12
Nice vs Angers - away @ 7.6- € 5
Rennes vs Nantes - away / Lille vs Auxerre - away @ 27.59- € 1
Rennes vs Nantes - away / Nice vs Angers - away @ 33.82- € 1
Bankroll used: 35.07 €


[{'match': 'Rennes vs Nantes - away', 'odd': 4.45, 'stake': 14},
 {'match': 'Monaco vs Strasbourg - away', 'odd': 5.1, 'stake': 3},
 {'match': 'Lille vs Auxerre - away', 'odd': 6.2, 'stake': 12},
 {'match': 'Nice vs Angers - away', 'odd': 7.6, 'stake': 5},
 {'match': 'Rennes vs Nantes - away / Lille vs Auxerre - away',
  'odd': 27.59,
  'stake': 1},
 {'match': 'Rennes vs Nantes - away / Nice vs Angers - away',
  'odd': 33.82,
  'stake': 1}]